# Proyecto Bimestral: Sistema de Recuperaci´on de Información basado en Reuters-21578
## 1. Introducción
El objetivo de este proyecto es diseñar, construir, programar y desplegar un Sistema de Recuperación de Información (SRI) utilizando el corpus Reuters-21578.
## 2. Fases del Proyecto
###  2.1. Adquisición de datos

*   Descargar el Corpus Reuters-21578
*   Descomprimir y organizar los archivos
*   Documentar el proceso de adquisición de datos

![Descripción de la imagen](images/corpus_reuters.jpg)

#### Descarga y análisis inicial del corpus

1. El archivo comprimido fue descargado y descomprimido en una carpeta local vinculada a un repositorio de GitHub.
2. Posteriormente, se analizó el contenido del corpus, obteniendo los siguientes resultados:
   - **Carpeta `test`**: Contiene 3019 archivos.
   - **Carpeta `training`**: Contiene 7769 archivos.
   - **Archivo `cats`**: Incluye las categorías.
   - **Archivo `readme`**: Proporciona información general sobre el corpus.
   - **Archivo `stopwords`**: Contiene una lista de palabras vacías.

#### Nota
Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install rarfile` para tener la biblioteca `rarfile` en nuestro entorno.

In [27]:
#!pip install rarfile

#### Nota
Al no encontrarnos en un entorno de Google Colab, sino en VS, nos basta con ejecutar una sola vez el comando `!pip install nltk` para tener la biblioteca `nltk` en nuestro entorno.

In [17]:
#!pip install nltk

In [1]:
# Librerias necesarias
import os
import re
import nltk
import rarfile
import subprocess
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### 2.2. Preprocesamiento

#### 2.2.1. Extraer el contenido relevante de los documentos

In [7]:
# Configurar la herramienta unrar
rarfile.UNRAR_TOOL = r"D:\UnRAR\UnRAR.exe"

# Ruta del archivo .rar
rar_path = 'material/reuters.rar'
output_dir = 'material/content'

# Crear el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Descomprimir el archivo .rar
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(output_dir)

# Verificar que se haya descomprimido correctamente
print("Archivos extraídos:")
print(os.listdir(output_dir))

Archivos extraídos:
['reuters']


In [15]:
# Directorios de documentos
train_dir = 'material/content/reuters/training'
test_dir = 'material/content/reuters/test'
cats_file = 'material/content/reuters/cats.txt'

# Diccionario para almacenar documentos
documentos = {}

# Función para extraer contenido de un archivo de noticias
def extraer_texto(filepath):
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            contenido = file.read()
            texto_limpio = contenido.strip()  # Elimina espacios en blanco iniciales y finales
            return texto_limpio
    except Exception as e:
        print(f"Error al leer el archivo {filepath}: {e}")
        return ""

# Función para cargar las categorías de los documentos
def cargar_categorias(filepath):
    categorias = {}
    try:
        with open(filepath, 'r', encoding='latin-1') as file:
            for linea in file:
                partes = linea.strip().split()  # Divide la línea en partes separadas por espacios
                if len(partes) >= 2:
                    doc_id = partes[0]  # Primer elemento es el id del documento (test/14826, training/1)
                    etiquetas = partes[1:]  # Resto de elementos son categorías
                    categorias[doc_id] = etiquetas
    except Exception as e:
        print(f"Error al leer el archivo de categorías {filepath}: {e}")
    return categorias

# Función para cargar los documentos y asociar categorías
def cargar_documentos(directorio, tipo, categorias_dict):
    archivos = os.listdir(directorio)
    if not archivos:
        print(f"No se encontraron archivos en {directorio}")
    for archivo in archivos:
        filepath = os.path.join(directorio, archivo)
        doc_id = f"{tipo}/{archivo}"
        texto = extraer_texto(filepath)
        categorias = categorias_dict.get(doc_id, [])  # Obtener categorías, si no hay, devuelve lista vacía

        # Almacenar en el diccionario
        documentos[doc_id] = {
            "texto": texto,
            "categorias": categorias
        }

In [16]:
# Cargar categorías
categorias_dict = cargar_categorias(cats_file)

# Cargar documentos de entrenamiento y prueba
cargar_documentos(train_dir, 'training', categorias_dict)
cargar_documentos(test_dir, 'test', categorias_dict)

# Verificar el tamaño y algunas muestras
print(f"Total de documentos cargados: {len(documentos)}")
if documentos:
    ejemplo_doc = list(documentos.items())[0]
    print(f"ID: {ejemplo_doc[0]}")
    print(f"Texto: {ejemplo_doc[1]['texto'][:500]}...")  # Mostrar los primeros 500 caracteres
    print(f"Categorías: {ejemplo_doc[1]['categorias']}")
else:
    print("No se cargaron documentos.")

Total de documentos cargados: 10788
ID: training/1
Texto: BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at th...
Categorías: ['cocoa']


#### 2.2.2. Realizar limpieza de datos: eliminación de caracteres no deseados, normalización de texto, etc.

#### 2.2.3. Tokenización: dividir el texto en palabras o tokens

#### 2.2.4. Eliminar stop words y  aplicar stemming o lematización.

### 2.3.  Representación de Datos en Espacio Vectorial